### WTP в Conjoint

WTP - willingness to pay

In [7]:
import pandas as pd
from IPython.display import display
import numpy as np

df = pd.read_csv('trailblazers.csv')

df.head(5)

,Attribute,Level,Partworth,Cost,Price
0,Number of Games,3-Game,0.03257,NaN,NaN
1,Number of Games,6-Game,0.24383,NaN,NaN
2,Number of Games,10-Game,-0.27640,NaN,NaN
3,Ticket Price,$15/seat/game,0.65646,10.0,15.0
4,Ticket Price,$25/seat/game,0.22011,12.0,25.0


## Functions

In [9]:
def get_diff(a):
    # разница между границами
    return max(a) - min(a)


def get_attr_importance(pw_diff, total_pw):
    # относительная важность
    return pw_diff / total_pw * 100


def get_attr_util(df):
    # расчет полезности (значимости) / utility
    df = df.copy()
    price_unit = None
    
    # расчет значимости составных частей для каждого продукта / уровня
    partworths = [r for r in df['Partworth']]
    _min = min(partworths)
    prices = [r for r in df['Price']]
    pw_diff = get_diff( partworths )
    
    # расчет полезности признака по уровень / продукт
    util = lambda pw: (pw - _min) if (pw != _min) else pw
    utilities = [util(pw) for pw in partworths]
    df['Utility'] = utilities
    
    if not np.isnan(prices[0]):
        price_diff = get_diff(prices)
        price_unit = price_diff / pw_diff
        
    df['PriceUnit'] = [price_unit for r in prices]
    
    return [df, pw_diff]


def get_attr_values(df, price_unit):
    # создаем таблицу полезности
    perc_values = []   
    get_perc_val = lambda u: u * price_unit
    perc_values = [get_perc_val(u) for u in df['Utility']]
    return perc_values

def get_wtp(df):
    # расчет WTP
    
    price_unit = None
    pw_total = 0.00
    attrs_list = []
    imp_data = []
    
    # группируем по атрибутам
    attrs = df.groupby(['Attribute'])
    attr_names = [a[0] for a in attrs]
    
    for attr in attrs:
        # расчет полезности каждого атрибута
        attr_df, pw_diff = get_attr_util(attr[1])
        pw_total += pw_diff
        attrs_list.append(attr_df)
        
        if 'PriceUnit' in attr_df.columns:
            price_unit = attr_df['PriceUnit'].values[0]
                 
    for attr, name in zip(attrs_list, attr_names):
        # определяем значение WTP относительно полезности его атрибута
        attr['WTP'] = get_attr_values(attr, price_unit)
        u_diff = get_diff( [u for u in attr['Utility'].values] )
        imp_data.append( (name, u_diff, get_attr_importance(u_diff, pw_total) ) )
    
    # итоговая таблица
    df_imp = pd.DataFrame(data=imp_data, columns=['Attribute', 'Utility', 'Importance'])\
                      .sort_values(by='Importance', ascending=False)
        
    return [attrs_list, df_imp]

### Расчет WTP

In [10]:
attrs, imp = get_wtp(df)

# таблица атрибутов и их полезности
display(imp)

,Attribute,Utility,Importance
3,Ticket Price,2.66160,60.291263
2,Ticket Location,2.47486,56.061179
1,Promo Item,0.81000,18.348333
0,Number of Games,0.79663,18.045472


In [11]:
# соединяем с WTP
merged = pd.concat(attrs).sort_index().sort_values("WTP", ascending = False).reset_index().drop('index', axis=1)
display(merged)

,Attribute,Level,Partworth,Cost,Price,Utility,PriceUnit,WTP
0,Ticket Location,200M,1.01148,40.00,NaN,1.74317,None,47.282237
1,Ticket Price,$15/seat/game,0.65646,10.00,15.0,1.65903,27.124283,45.000000
2,Ticket Price,$25/seat/game,0.22011,12.00,25.0,1.22268,27.124283,33.164319
3,Ticket Location,300C,0.43716,12.00,NaN,1.16885,None,31.704219
4,Ticket Price,$35/seat/game,0.12600,18.00,35.0,1.12857,27.124283,30.611653
5,Ticket Location,300M,0.15736,18.00,NaN,0.88905,None,24.114844
6,Number of Games,6-Game,0.24383,NaN,NaN,0.52023,None,14.110866
7,Promo Item,Dog n Pop,0.17428,3.25,NaN,0.49214,None,13.348945
8,Promo Item,Priority Playoff Tickets,0.12511,0.00,NaN,0.44297,None,12.015244
9,Promo Item,$20 Certificate,0.01689,10.00,NaN,0.33475,None,9.079854


In [ ]:
pivot = merged.T
display(pivot)

In [12]:
# посмотрим, какое Промо наиболее значимо
merged[merged['Attribute'] == 'Promo Item']

,Attribute,Level,Partworth,Cost,Price,Utility,PriceUnit,WTP
7,Promo Item,Dog n Pop,0.17428,3.25,NaN,0.49214,None,13.348945
8,Promo Item,Priority Playoff Tickets,0.12511,0.00,NaN,0.44297,None,12.015244
9,Promo Item,$20 Certificate,0.01689,10.00,NaN,0.33475,None,9.079854
10,Promo Item,Apparel,0.00158,12.00,NaN,0.31944,None,8.664581
13,Promo Item,No Promo,-0.31786,0.00,NaN,-0.31786,None,-8.621725
